In [2]:
%matplotlib inline
import os
os.environ['PY3_PROD'] = '1'
%load_ext autoreload
%autoreload 2

os.system('kinit')

1

In [4]:
import numpy as np
import math
import pandas as pd
from typing import Union, List
import datetime
from pycmqlib3.utility import misc
from pycmqlib3.utility.dbaccess import *
import pycmqlib3.analytics.data_handler as dh
import pathlib
from pycmqlib3.utility.process_wt_data import *
from wtpy.wrapper import WtDataHelper
# from pycmqlib3.utility import dataseries
# from pycmqlib3.analytics.tstool import *
# from pycmqlib3.analytics.btmetrics import *


In [5]:
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 150)

In [6]:
from misc_scripts.daily_update_job import update_data_from_xl
update_data_from_xl(lookback=19570)

C:\dev\akshare\akshare\__init__.py:2637: UserWarning: 为了支持更多特性，请将 Pandas 升级到 2.1.0 及以上版本！
  warnings.warn(


saving data for ifind_data.xlsx:ferrous_d, total cols:157
saving data for ifind_data.xlsx:base_d, total cols:172
saving data for ifind_data.xlsx:macro_d, total cols:80
saving data for ifind_data.xlsx:ferrous_w, total cols:148
saving data for ifind_data.xlsx:base_w, total cols:77
saving data for ifind_data.xlsx:const_d, total cols:48
saving data for ifind_data.xlsx:petchem_d, total cols:133


In [9]:
base_folder = "C:/dev/wtdev/storage/his"

zip_wt_dir(base_folder, 'C:/dev/data/wt_data_230626.zip', cutoff=datetime.date(2023,6,20))

In [7]:
base_folder = "C:/dev/wtdev/storage/his"
update_folder = "C:/dev/data/his"
cutoff = datetime.date(2024,1,20)
update_wt_store(base_folder=base_folder, update_folder = update_folder, cutoff=cutoff)

day-CFFEX
day-DCE
day-CZCE
day-SHFE
day-INE
day-GFEX
min1-CFFEX
min1-DCE
min1-CZCE
min1-SHFE
min1-INE
min1-GFEX
min5-CFFEX
min5-DCE
min5-CZCE
min5-SHFE
min5-INE
min5-GFEX


In [5]:
from misc_scripts.aks_data_update import *
start_date=datetime.date(2023,12,1)
end_date=datetime.date(2023,12,11)
update_exch_receipt_table(start_date, end_date, flavor='mysql')

C:\dev\akshare\akshare\__init__.py:2625: UserWarning: 为了支持更多特性，请将 Pandas 升级到 2.1.0 及以上版本！
  warnings.warn(


2023-12-11


ImportError: lxml not found, please install it

In [7]:
from misc_scripts.fun_factor_update import *
tday = datetime.date(2023, 7, 28)
update_fun_factor(run_date=tday)

        index         0
0  2023-04-28  0.696535
1  2023-05-04 -0.925318
2  2023-05-05 -0.103820
3  2023-05-08  0.983475
4  2023-05-09  0.903388
5  2023-05-10  0.994595
6  2023-05-11  0.770550
7  2023-05-12  0.419891
8  2023-05-15  0.701273
9  2023-05-16  0.775955
10 2023-05-17  0.850637
11 2023-05-18  0.247778
12 2023-05-19  0.701273
13 2023-05-22  0.124873
14 2023-05-23  0.402547
15 2023-05-24 -0.090935
16 2023-05-25 -0.246304
17 2023-05-26 -0.456392
18 2023-05-29 -0.278023
19 2023-05-30 -0.543659
20 2023-05-31 -0.845395
21 2023-06-01 -0.336042
22 2023-06-02 -0.004972
23 2023-06-05  0.861163
24 2023-06-06  0.274236
25 2023-06-07 -0.731643
26 2023-06-08 -0.954064
27 2023-06-09 -0.182363
28 2023-06-12 -0.741360
29 2023-06-13  0.197821
30 2023-06-14  0.345182
31 2023-06-15 -0.056931
32 2023-06-16  0.569035
33 2023-06-19  0.146815
34 2023-06-20 -0.196892
35 2023-06-21 -0.075236
36 2023-06-26 -0.988880
37 2023-06-27 -0.546505
38 2023-06-28 -0.845231
39 2023-06-29 -1.000000
40 2023-06-30 -0

In [ ]:
start_date = datetime.date(2022,12,22)
end_date = datetime.date(2022,12,23)
src_folder = "C:/dev/wtdev/storage/his"
dst_folder = "C:/dev/wtdev/storage/his"
cutoff_date = 20221223

save_bars_to_wt_store(exchange_list=['GFEX'],
                        start_date=start_date,
                        end_date=end_date,
                        cutoff_date=cutoff_date,
                        process_min=True,
                        process_day=True,
                        src_folder=src_folder,
                        dst_folder=dst_folder)

In [ ]:
start_date = datetime.date(2022,12,22)
end_date = datetime.date(2022,12,22)
dst_folder = "C:/dev/wtdev/storage/his"

save_ticks_to_wt_store(
        exchange_list=['GFEX'],
        start_date=start_date,
        end_date=end_date,
        dst_folder=dst_folder,
        exclusion_list=[])

In [4]:
period = 'day'
exch = 'CZCE'
cont = 'ME1203'
src_folder = 'c:/dev/wtdev/storage/his'
filename = '%s/%s/%s/%s.dsb' % (src_folder, period, exch, cont)
dtHelper = WtDataHelper()
curr_df = dtHelper.read_dsb_bars(filename).to_df()
curr_df

Exception ignored on calling ctypes callback function: <bound method WtDataHelper.on_log_output of <wtpy.wrapper.WtDtHelper.WtDataHelper object at 0x000001AF6CB18310>>
Traceback (most recent call last):
  File "C:\dev\wtpy\wtpy\wrapper\WtDtHelper.py", line 40, in on_log_output
    message = bytes.decode(message, 'utf-8')
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd5 in position 0: invalid continuation byte
Exception ignored on calling ctypes callback function: <bound method WtDataHelper.on_log_output of <wtpy.wrapper.WtDtHelper.WtDataHelper object at 0x000001AF6CB18310>>
Traceback (most recent call last):
  File "C:\dev\wtpy\wtpy\wrapper\WtDtHelper.py", line 40, in on_log_output
    message = bytes.decode(message, 'utf-8')
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xb6 in position 44: invalid start byte


,date,open,high,low,close,settle,money,volume,hold,diff,bartime
199000000000,20111028,3199.0,3220.0,3130.0,3139.0,3157.0,6.394188e+09,40508.0,3342.0,0.0,199000000000
199000000000,20111031,3136.0,3198.0,3110.0,3134.0,3144.0,5.490682e+09,34928.0,3646.0,304.0,199000000000
199000000000,20111101,3140.0,3154.0,3125.0,3131.0,3137.0,2.094575e+09,13354.0,3052.0,-594.0,199000000000
199000000000,20111102,3112.0,3128.0,3095.0,3123.0,3114.0,1.200758e+09,7712.0,3482.0,430.0,199000000000
199000000000,20111103,3120.0,3122.0,3107.0,3114.0,3117.0,5.155518e+08,3308.0,3006.0,-476.0,199000000000
199000000000,20111104,3126.0,3145.0,3122.0,3139.0,3132.0,9.170496e+08,5856.0,2726.0,-280.0,199000000000
199000000000,20111107,3139.0,3142.0,3121.0,3126.0,3130.0,2.735620e+08,1748.0,2640.0,-86.0,199000000000
199000000000,20111108,3130.0,3134.0,3114.0,3118.0,3123.0,2.573352e+08,1648.0,2774.0,134.0,199000000000
199000000000,20111109,3125.0,3128.0,3008.0,3019.0,3042.0,3.405215e+09,22388.0,6810.0,4036.0,199000000000
199000000000,20111110,2975.0,2975.0,2920.0,2920.0,2937.0,1.887904e+09,12856.0,8186.0,1376.0,199000000000


In [12]:
pd.concat([curr_df, curr_df])


,date,open,high,low,close,settle,money,volume,hold,diff,bartime
199000000000,20230117,4001.0,4027.0,3973.0,4011.0,3996.0,4.148817e+07,1038.0,573.0,573.0,199000000000
199000000000,20230118,4027.0,4051.0,4014.0,4037.0,4036.0,1.267393e+07,314.0,713.0,140.0,199000000000
199000000000,20230119,4043.0,4085.0,4043.0,4080.0,4066.0,1.431537e+07,352.0,821.0,108.0,199000000000
199000000000,20230120,4075.0,4076.0,4050.0,4070.0,4065.0,2.040859e+07,502.0,944.0,123.0,199000000000
199000000000,20230130,4127.0,4157.0,4080.0,4084.0,4123.0,4.090941e+07,992.0,1201.0,257.0,199000000000
...,...,...,...,...,...,...,...,...,...,...,...
199000000000,20231120,3928.0,3995.0,3911.0,3983.0,3952.0,6.565526e+10,1661178.0,1622469.0,44375.0,199000000000
199000000000,20231121,4000.0,4053.0,3983.0,3983.0,4019.0,8.158945e+10,2030063.0,1628588.0,6118.0,199000000000
199000000000,20231122,3986.0,4017.0,3955.0,4001.0,3986.0,8.056833e+10,2020789.0,1599519.0,-29070.0,199000000000
199000000000,20231123,4003.0,4016.0,3896.0,3934.0,3950.0,9.559915e+10,2420217.0,1472799.0,-126720.0,199000000000


In [10]:
df = curr_df[['date', 'reserve', 'time', 'open', 'high', 'low', 'close', 'settle', 'money', 'volume', 'hold', 'diff']]
folder_loc = 'c:/data'
period = 'day'
contract = cont
filename = f'{folder_loc}/{contract}.dsb'
print(filename)
dtHelper = WtDataHelper()

BUFFER = WTSBarStruct*len(df)
buffer = BUFFER()
df.apply(assign, buffer=buffer)
#pathlib.Path(folder_loc).mkdir(parents=True, exist_ok=True)
dtHelper.store_bars(barFile=filename, firstBar=buffer, count=len(df), period=period)

#save_bars_to_dsb(curr_df, cont, folder_loc = 'c:/dev/data', period = period)

c:/data/rb2401.dsb


0

In [25]:
df

,date,reserve,time,open,high,low,close,settle,money,volume,hold,diff
199000000000,20230117,0,0,4001.0,4027.0,3973.0,4011.0,3996.0,4.148817e+07,1038.0,573.0,573.0
199000000000,20230118,0,0,4027.0,4051.0,4014.0,4037.0,4036.0,1.267393e+07,314.0,713.0,140.0
199000000000,20230119,0,0,4043.0,4085.0,4043.0,4080.0,4066.0,1.431537e+07,352.0,821.0,108.0
199000000000,20230120,0,0,4075.0,4076.0,4050.0,4070.0,4065.0,2.040859e+07,502.0,944.0,123.0
199000000000,20230130,0,0,4127.0,4157.0,4080.0,4084.0,4123.0,4.090941e+07,992.0,1201.0,257.0
...,...,...,...,...,...,...,...,...,...,...,...,...
199000000000,20231120,0,0,3928.0,3995.0,3911.0,3983.0,3952.0,6.565526e+10,1661178.0,1622469.0,44375.0
199000000000,20231121,0,0,4000.0,4053.0,3983.0,3983.0,4019.0,8.158945e+10,2030063.0,1628588.0,6118.0
199000000000,20231122,0,0,3986.0,4017.0,3955.0,4001.0,3986.0,8.056833e+10,2020789.0,1599519.0,-29070.0
199000000000,20231123,0,0,4003.0,4016.0,3896.0,3934.0,3950.0,9.559915e+10,2420217.0,1472799.0,-126720.0


In [ ]:
sdate = datetime.date(2000,1,1)
edate = datetime.date(2022,9,30)

cnx = connect(**dbconfig)
cont = 'TA301'
mdf = load_min_data_to_df(cnx, 'fut_min', cont, sdate, edate, index_col = None, shift_datetime = True)

# mdf['rdate'] = mdf['date'].astype('datetime64').dt.strftime('%Y%m%d').astype('int64')
# mdf['time'] = 0
# mdf['hold'] = mdf['openInterest']
# mdf['diff'] = mdf['hold'].diff().fillna(0)
# mdf['money'] = mdf['close'] * mdf['volume']
# mdf['vol'] = mdf['volume']
col_list = ['date', 'time', 'open', 'high', 'low', 'close', 'money', 'vol', 'hold', 'diff']
# mdf = mdf[col_list]            
mdf = conv_min_data(mdf)
print(mdf[col_list])

In [14]:
sdate = datetime.date(2000,1,1)
edate = datetime.date(2022,9,30)

cnx = connect(**dbconfig)
cont = 'TA301'
mdf = load_min_data_to_df(cnx, 'fut_min', cont, sdate, edate, index_col = None, shift_datetime = True)

# mdf['rdate'] = mdf['date'].astype('datetime64').dt.strftime('%Y%m%d').astype('int64')
# mdf['time'] = 0
# mdf['hold'] = mdf['openInterest']
# mdf['diff'] = mdf['hold'].diff().fillna(0)
# mdf['money'] = mdf['close'] * mdf['volume']
# mdf['vol'] = mdf['volume']
col_list = ['date', 'time', 'open', 'high', 'low', 'close', 'money', 'vol', 'hold', 'diff']
# mdf = mdf[col_list]            
mdf = conv_min_data(mdf)
print(mdf[col_list])

           date        time    open    high     low   close     money  vol  \
0      20120120  2201201015  8780.0  8780.0  8780.0  8780.0   17560.0    2   
1      20120130  2201300901  8824.0  8824.0  8824.0  8824.0   17648.0    2   
2      20120131  2201311008  9016.0  9016.0  9016.0  9016.0   18032.0    2   
3      20120131  2201311046  9016.0  9070.0  9016.0  9070.0   18140.0    2   
4      20120131  2201311055  9070.0  9070.0  9066.0  9066.0   18132.0    2   
...         ...         ...     ...     ...     ...     ...       ...  ...   
38845  20130115  2301151001  8874.0  9000.0  8874.0  9000.0   36000.0    4   
38846  20130115  2301151103  9000.0  9022.0  9000.0  9022.0   36088.0    4   
38847  20130116  2301161500  9022.0  9022.0  8850.0  8850.0  141600.0   16   
38848  20130117  2301170929  9078.0  9078.0  9078.0  9078.0  108936.0   12   
38849  20130117  2301171037  9078.0  9078.0  8906.0  8906.0   35624.0    4   

       hold   diff  
0         2    0.0  
1         2    0.0  


In [ ]:
ddf = load_daily_data_to_df(cnx, 'fut_daily', cont, datetime.date(2000,1,1), edate, index_col='date', field = 'instID', date_as_str = False)
ddf = ddf.rename(columns = {'openInterest': 'open_interest'}).reset_index()
ddf['pre_settle'] = ddf['settle'].shift(1)
ddf['pre_interest'] = ddf['open_interest'].shift(1)
ddf['pre_close'] = ddf['close'].shift(1)
ddf['date'] = ddf['date'].astype('datetime64').dt.strftime('%Y%m%d').astype('int64')
ddf = ddf.set_index('date')

In [29]:
from wtpy.wrapper import WtDataHelper
from wtpy.WtCoreDefs import WTSBarStruct, WTSTickStruct
from ctypes import POINTER
from wtpy.SessionMgr import SessionMgr
import pandas as pd

dtHelper = WtDataHelper()

#def test_store_bars():
df = pd.read_csv('C:/dev/wtpy/demos/storage/csv/CFFEX.IF.HOT_m5.csv')
df = df.rename(columns={
    '<Date>':'date',
    ' <Time>':'time',
    ' <Open>':'open',
    ' <High>':'high',
    ' <Low>':'low',
    ' <Close>':'close',
    ' <Volume>':'vol',
    })
df['date'] = df['date'].astype('datetime64').dt.strftime('%Y%m%d').astype('int64')
df['time'] = (df['date']-19900000)*10000 + df['time'].str.replace(':', '').str[:-2].astype('int')

df

,date,time,open,high,low,close,vol
0,20140102,2401020935,2342.2,2343.4,2339.2,2343.2,10226
1,20140102,2401020940,2343.4,2343.8,2336.6,2338.0,12114
2,20140102,2401020945,2338.2,2344.6,2338.2,2343.2,13029
3,20140102,2401020950,2343.4,2344.4,2339.2,2339.2,10843
4,20140102,2401020955,2339.2,2339.8,2335.2,2335.2,11218
...,...,...,...,...,...,...,...
68140,20191031,2910311440,3877.8,3878.8,3865.0,3867.2,2442
68141,20191031,2910311445,3867.2,3873.2,3867.2,3869.4,1422
68142,20191031,2910311450,3870.0,3873.8,3870.0,3873.4,923
68143,20191031,2910311455,3873.0,3874.0,3869.6,3872.4,1236


In [30]:
BUFFER = WTSBarStruct*len(df)
buffer = BUFFER()

def assign(procession, buffer):
    tuple(map(lambda x: setattr(buffer[x[0]], procession.name, x[1]), enumerate(procession)))


df.apply(assign, buffer=buffer)
print(df)
print(buffer[0].to_dict)
print(buffer[-1].to_dict)

dtHelper.store_bars(barFile="C:/dev/wtpy/demos/storage/CFFEX.IF.HOT_m5.dsb", firstBar=buffer, count=len(df), period="m5")

           date        time    open    high     low   close    vol
0      20140102  2401020935  2342.2  2343.4  2339.2  2343.2  10226
1      20140102  2401020940  2343.4  2343.8  2336.6  2338.0  12114
2      20140102  2401020945  2338.2  2344.6  2338.2  2343.2  13029
3      20140102  2401020950  2343.4  2344.4  2339.2  2339.2  10843
4      20140102  2401020955  2339.2  2339.8  2335.2  2335.2  11218
...         ...         ...     ...     ...     ...     ...    ...
68140  20191031  2910311440  3877.8  3878.8  3865.0  3867.2   2442
68141  20191031  2910311445  3867.2  3873.2  3867.2  3869.4   1422
68142  20191031  2910311450  3870.0  3873.8  3870.0  3873.4    923
68143  20191031  2910311455  3873.0  3874.0  3869.6  3872.4   1236
68144  20191031  2910311500  3872.4  3879.0  3872.4  3879.0   2648

[68145 rows x 7 columns]
{'date': 20140102, 'reserve': 0, 'time': 2401020935, 'open': 2342.2, 'high': 2343.4, 'low': 2339.2, 'close': 2343.2, 'settle': 0.0, 'money': 0.0, 'vol': 10226.0, 'hold': 

1

In [31]:
#test_store_bars()

filename = 'C:/dev/wtpy/demos/storage/CFFEX.IF.HOT_m5.dsb'
dtHelper = WtDataHelper()
curr_df = dtHelper.read_dsb_bars(filename).to_df()
curr_df


,date,open,high,low,close,settle,money,volume,hold,diff,bartime
201401020935,20140102,2342.2,2343.4,2339.2,2343.2,0.0,0.0,10226.0,0.0,0.0,201401020935
201401020940,20140102,2343.4,2343.8,2336.6,2338.0,0.0,0.0,12114.0,0.0,0.0,201401020940
201401020945,20140102,2338.2,2344.6,2338.2,2343.2,0.0,0.0,13029.0,0.0,0.0,201401020945
201401020950,20140102,2343.4,2344.4,2339.2,2339.2,0.0,0.0,10843.0,0.0,0.0,201401020950
201401020955,20140102,2339.2,2339.8,2335.2,2335.2,0.0,0.0,11218.0,0.0,0.0,201401020955
...,...,...,...,...,...,...,...,...,...,...,...
201910311440,20191031,3877.8,3878.8,3865.0,3867.2,0.0,0.0,2442.0,0.0,0.0,201910311440
201910311445,20191031,3867.2,3873.2,3867.2,3869.4,0.0,0.0,1422.0,0.0,0.0,201910311445
201910311450,20191031,3870.0,3873.8,3870.0,3873.4,0.0,0.0,923.0,0.0,0.0,201910311450
201910311455,20191031,3873.0,3874.0,3869.6,3872.4,0.0,0.0,1236.0,0.0,0.0,201910311455
